In [1]:
from nltk import sent_tokenize
from nltk import word_tokenize
from threading import Thread
from keybert import KeyBERT
from pprint import pprint
import pandas as pd
import time

In [2]:
path_dict = 'demo_test/TITLE.csv'  #  Путь в словарю
path_text ='text.txt' # Путь к тексту

In [3]:
data = set(map(lambda word: word.lower(), list(pd.read_csv(path_dict).Word)))
model = KeyBERT('distilbert-base-nli-mean-tokens')
result = {}
key_words = set()

In [4]:
class FindThread(Thread):
    
    def __init__(self, word, sent_index, word_index):
        Thread.__init__(self)
        self.word = word
        self.sent_index = sent_index
        self.word_index = word_index 
        
    
    def run(self):
        
        if self.word in data:
            update_dict(dict_=result, key=self.word, value=[self.sent_index, self.word_index] )

In [5]:
def main(text : str, sent_index) -> None:
    
    for index_word, word in enumerate(word_tokenize(text)):
       
        thread = FindThread(word, sent_index, index_word)
        thread.start()

In [6]:
def update_dict(dict_ : dict, key : str, value) -> None:
    el = dict_.get(key, 'error')
    if el == 'error':
        dict_.update({key : [value]})
    else:
        el.append(value)

In [7]:
def common_words(key_words):
    return [el for el in key_words if el in data ]

In [8]:
with open(path_text, 'r') as code_book:
        sent_index = 0
        start_time = time.time()
        for sent in sent_tokenize(code_book.read()):
            
            try:
                tmp = model.extract_keywords(sent.lower(), keyphrase_length=1, use_maxsum=True)   
                key_words = key_words.union(set(tmp))
                
            except Exception as e:
                e = e
                
            main(sent, sent_index)
            sent_index += 1
            print("--- %s seconds ---" % (time.time() - start_time))
        
        pprint(result)

--- 2.5559675693511963 seconds ---
--- 2.635148525238037 seconds ---
--- 2.685586929321289 seconds ---
--- 2.7300939559936523 seconds ---
--- 2.7727434635162354 seconds ---
--- 2.858961343765259 seconds ---
--- 3.0373237133026123 seconds ---
{'asset': [[0, 32], [6, 24]],
 'beta': [[2, 25]],
 'better': [[5, 26], [6, 22]],
 'capital': [[1, 24]],
 'capitalization': [[5, 6]],
 'commodity': [[0, 16]],
 'determine': [[0, 5]],
 'estimates': [[5, 27]],
 'expected': [[1, 15]],
 'generate': [[1, 21], [2, 23]],
 'get': [[5, 25]],
 'independent': [[4, 16]],
 'index': [[2, 12], [3, 12]],
 'influence': [[0, 26]],
 'interest': [[0, 18]],
 'many': [[0, 7]],
 'market': [[3, 10], [4, 22], [5, 5]],
 'model': [[3, 22], [5, 23], [6, 20]],
 'movement': [[0, 29]],
 'multiple': [[6, 17]],
 'named': [[6, 10]],
 'often': [[0, 2]],
 'particular': [[0, 21], [2, 28]],
 'premium': [[4, 24]],
 'price': [[0, 13], [0, 28]],
 'project': [[1, 13]],
 'question': [[4, 6]],
 'regression': [[1, 6], [6, 19]],
 'relation': [[

In [9]:
common_words(key_words=key_words)

['regression',
 'asset',
 'variable',
 'better',
 'stock',
 'beta',
 'price',
 'market',
 'risk',
 'movement',
 'premium',
 'capital',
 'expected',
 'stocks',
 'independent',
 'model',
 'project']